##Midterm

In [6]:
import pandas as pd
import numpy as np
import csv as csv
import scipy
from sklearn.ensemble import RandomForestClassifier
from patsy import dmatrices
import sklearn.cross_validation as cv
import matplotlib.pyplot as plt 
%matplotlib inline
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold, train_test_split

# loading input files, test.csv is the same as train.csv without the Survived column
train_data = "train.csv"
test_data = "test.csv"
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

NameError: name 'train_path' is not defined

In [ ]:
train_data.info()

In [ ]:
# clean the data for genders
train_data['Gender'] = train_data['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
test_data['Gender'] = test_data['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [ ]:
# fill data for missing Age values using median of gender/class group in both test/train data
median_ages = np.zeros(3)
for i in range(1,4):
    median_ages[i-1] = train_data[train_data["Pclass"] == i]["Age"].dropna().median()

for i in range(1,4):
    train_data.loc[(train_data["Age"].isnull()) & (train_data["Pclass"] == i), "Age"] = median_ages[i-1]
    test_data.loc[(test_data["Age"].isnull()) & (test_data["Pclass"] == i), "Age"] = median_ages[i-1]

In [ ]:
# transform the age variable into a discrete variable, binned in increments of 10
train_data['Age_discrete'] = train_data['Age'].apply(lambda x: int(x - (x % 10)))

In [ ]:
# transform the fare variable into a discrete variable, binned in increments of 25
train_data['Fare_discrete'] = train_data['Fare'].apply(lambda x: int(x - (x % 25)))

In [ ]:
# drop all string variables
train_data = train_data.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age', 'Fare'], axis=1)

In [ ]:
#new train_data with removed variables and missing values filled in for age
train_data.info()

In [ ]:
#repeat for test_data for removing string variable
test_data.info()

In [ ]:
#replace NaN values in fare using the median 
#(missing age values have been replaced in code above) 
if len(test_data.Fare[ test_data.Fare.isnull() ]) > 0:
    median_fares = np.zeros(3)
for fare in range(0,3): # loop 0 to 2
    median_fares[fare] = test_data[ test_data.Pclass == fare+1 ]['Fare'].dropna().median()
for fare in range(0,3): # loop 0 to 2
    test_data.loc[ (test_data.Fare.isnull()) & (test_data.Pclass == fare+1 ), 'Fare'] = median_fares[fare]

In [ ]:
# transform the age variable into a discrete variable, binned in increments of 10
test_data['Age_discrete'] = test_data['Age'].apply(lambda x: int(x - (x % 10)))

In [ ]:
# transform the fare variable into a discrete variable, binned in increments of 25
test_data['Fare_discrete'] = test_data['Fare'].apply(lambda x: int(x - (x % 25)))

In [ ]:
# drop all string variables
test_data = test_data.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age', 'Fare'], axis=1)

In [ ]:
#new test_data with removed variables and missing values filled for age and fare
test_data.info()

In [ ]:
train_data_np = train_data.astype(np.int32).values
X= train_data_np[:,:-1]
y=train_data_np[:,-1]

In [ ]:
#We split X and y into train and test datasets
(X_train_data, X_test_data, y_train_data, y_test_data) = cv.train_test_split(X, y, test_size=.05)

In [ ]:
#We instanciate the classifier
logreg = LogisticRegression()

In [ ]:
#Train model and we get predicted values on the test set
logreg.fit(X_train_data, y_train_data)
y_predicted = logreg.predict(X_test_data)

In [ ]:
#outcomes
plt.imshow(np.vstack((y_test_data, y_predicted)), 
interpolation='none', cmap='bone')
plt.yticks([]);
plt.title(("Actual and predicted survival" "outcomes on the test set"))

In [ ]:
cv.cross_val_score(logreg, X, y)

In [ ]:
res.summary()

In [ ]:
output = res.predict(test_data_x)
output = np.asanyarray(output).ravel()
output = np.round(output)

In [ ]:
output_file = open("Logit.csv", "wb")
output_file_object = csv.writer(output_file)
output_file_object.writerow(["PassengerId", "Survived"])
output_file_object.writerows(zip(test_data["PassengerId"], output))
output_file.close()

In [ ]:
# IMPLEMENTING K-FOLD CLASSIFICATION

In [ ]:
# separating predictor columns from response columns
x_train_data = train_data[train_data.columns.values[1:]].as_matrix()
response_train_data= train_data.Survived.as_matrix()

In [ ]:
# use sklearn to create indices on which to split for each fold
tenFold = StratifiedKFold(response_train_data, n_folds=10)
cCorrect = 0
cTotal = 0

In [ ]:
# iterate through each set of indices
for i, (tr,te) in enumerate(tenFold):
    x_tr = x_train_data[tr]
    out_tr = response_train_data[tr]
    x_te = x_train_data[te]
    out_te = response_train_data[te]
# construct new classifier each time
model = RandomForestClassifier()
model.fit(x_tr, out_tr)
out_pred = model.predict(x_te)
correct, total = (out_te == out_pred).sum(), len(out_te)
# increment accuracy values with each iteration
cCorrect += correct
cTotal += total
print '10-fold accuracy: %f' % (cCorrect * 1.0 / cTotal)
print "\n"

In [ ]:
model = RandomForestClassifier()
model.fit(x_train_data, response_train_data)

In [ ]:
# use pandas to iterate through feature combinations and return the best one
fRank = pd.Series(model.feature_importances_, index=train_data.columns.values[1:]).order(ascending=False)
print fRank
print "\n"

In [ ]:
# tuning attributes we are interested in testing
options = {
"n_estimators": [100],
"criterion": ['gini', 'entropy'],
'max_features': [1.0, .50, 'sqrt'],
'max_depth': [3,4,5,6,7,8, None],
'min_samples_split': [1,2],
'min_samples_leaf': [1,2]
    }

In [ ]:
model = RandomForestClassifier()
tuning = GridSearchCV(model, options, cv=12, verbose=0)
tuning.fit(x_train_data, response_train_data)


In [ ]:
print tuning.best_score_
print tuning.best_params_

In [ ]:
# fit the forest using best tuning attributes
forest = RandomForestClassifier(n_estimators = 100, criterion='entropy', max_features='sqrt', \
max_depth=6, min_samples_leaf = 1, min_samples_split = 2)
forest = forest.fit(train_dataValues[0::,1::],train_dataValues[0::,0])

In [ ]:
# predict the test data
predictions = forest.predict(test_dataValues).astype(int)

In [ ]:
output_file = open("RFC.csv", "wb")
output_file_object = csv.writer(output_file)
output_file_object.writerow(["PassengerId", "Survived"])
output_file_object.writerows(zip(test_data["PassengerId"], output))
output_file.close()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
 
# shuffle and split training and test sets
X_train_data, X_test_data, y_train_data, y_test_data = train_test_split(X, y, test_size=.25)
forest.fit(X_train_data, y_train_data)

In [ ]:
 # Determine the false positive and true positive rates
fpr, tpr,_ = roc_curve(y_test_data, forest.predict(X_test_data)[0::,0])

In [ ]:
# Calculate the AUC
roc_auc = auc(fpr, tpr)
print 'ROC AUC: %0.2f' % roc_auc

In [5]:
# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.savefig('ROC.png')

NameError: name 'fpr' is not defined